<a href="https://colab.research.google.com/github/danielmohansahu/M3DETR/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M3DETR Evaluation Notebook

This notebook demonstrates evaluation of the [M3DETR](https://arxiv.org/pdf/2104.11896.pdf) 3D Object Detection model architecture and training pipeline. Specifically it supports training and inference of [Group 13's fork](https://github.com/danielmohansahu/M3DETR). All training and evaluation uses the [KITTI 3D Detection Evaluation Dataset](https://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d).

Note - this notebook requires memory resources beyond those available in the current free tier of Google Colab. In order to run this script you will need to have a Google Colab Pro account (or possibly connect to your personal upgraded Google Drive).

In [ ]:
# sanity check hardware
!nvidia-smi

# sanity check required packages are installed
import torch
import torchvision
from tqdm import tqdm

# install M3DETR dependencies
!pip install spconv-cu120 cython open3d

# install dataset management helper
!python3 -m pip install fiftyone
!python3 -m pip install fiftyone-db-ubuntu2204

# import dataset management helper
import fiftyone

In [ ]:
# clone repository
!git clone https://github.com/danielmohansahu/M3DETR.git -b feature/sahu/profiling
%cd M3DETR

# build repository
!python3 setup.py develop

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

# sync with cached data
import pathlib
import shutil

# local location of drive cache (mounted)
cache_file = pathlib.Path("/content/drive/MyDrive/cache/m3detr/kitti.zip")

# local location of dataset
data_dir = pathlib.Path("/content/M3DETR/data/kitti")

if not cache_file.exists():
  # download and fully process KITTI from scratch. this will be slow!

  print("Downloading and processing KITTI dataset...")
  dataset = fiftyone.zoo.load_zoo_dataset('kitti-multiview', dataset_dir="data/kitti")

  # rename folders to follow convention
  print("Renaming and removing unneeded files / folders...")
  !cd data/kitti && mv test testing && mv train training

  # remove those we don't care about
  !cd data/kitti && rm -rf tmp-download testing/right training/right

  # post-process via OpenPCD scripts
  print("Post-processing dataset for OpenPCD...")
  !python3 -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos \
              tools/cfgs/dataset_configs/kitti_dataset.yaml

  # zip up the results for faster copy speeds
  print("Archiving dataset...")
  !zip kitti.zip /content/M3DETR/data/kitti -r

  # finally, copy over results to drive
  print("Backing up to drive...")
  !mv kitti.zip /content/drive/MyDrive/cache/m3detr/kitti.zip

elif not (data_dir / "training").is_dir() or not (data_dir / "testing").is_dir():
  # pull pre-processed KITTI dataset from local drive

  print("Copying over cached KITTI dataset...")
  !cp /content/drive/MyDrive/cache/m3detr/kitti.zip /content/M3DETR/data/

  print("Unzipping cached KITTI dataset...")
  !cd /content/M3DETR/data && unzip kitti.zip

# sanity check before continuing
assert (data_dir / "training").is_dir(), "Missing training data!"
assert (data_dir / "testing").is_dir(), "Missing testing data!"
print("KITTI dataset loaded.")